In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib
import ipdb
import matplotlib.pyplot as plt
# matplotlib.use('TkAgg')

#TO-DO: Define LSTMNet
""" Class for defining the LSTM Network """
class LSTMNet(nn.Module):
    def __init__(self):
        """" Initialize the network here. You can use a combination of nn.LSTMCell and nn.Linear. 
        Number of layers and hidden size is upto you. Hint: A network with less than 3 layers and 
        64 dimensionality should suffice.
        """
        s
        super(LSTMNet, self).__init__()
        self.hidden_dim=64 #taking hidden dimension=64
        self.lstm1=nn.LSTMCell(1,self.hidden_dim) #defining the LSTM  unit1
        self.lstm2=nn.LSTMCell(self.hidden_dim,self.hidden_dim) #defining the second layer LSTM unit2
        self.linear=nn.Linear(self.hidden_dim,1) #fully connected layer
        pass
 
    def forward(self, obs_seq, pred_len = 0):
        """ This function takes the input sequence and predicts the output sequence. 
            Arguments:
                input_seq (torch.Tensor) : Input sequence with shape <batch size x sequence length x number of dimensions>
                pred_len (int) : Length of the sequence to be predicted.

        Hint: There should be 2 for loops. First one iterates over the observed sequence. Second one predicts the output sequence
        one step at a time.

        """
        output=[]
        h_t=torch.zeros(obs_seq.size(0),self.hidden_dim,dtype=torch.double,requires_grad=False) #deifing the LSTM cell output variable
        c_t=torch.zeros(obs_seq.size(0),self.hidden_dim,dtype=torch.double,requires_grad=False) #defining initial cell state variable
        h_t2=torch.zeros(obs_seq.size(0),self.hidden_dim,dtype=torch.double,requires_grad=False) #defining the LSTM cell 2 output variable
        c_t2=torch.zeros(obs_seq.size(0),self.hidden_dim,dtype=torch.double,requires_grad=False) #definnig final cell state variable for LST cell 
        for i,input_seq in enumerate(obs_seq.chunk(obs_seq.size(1),dim=1)): #iterating over the columns
            h_t,c_t=self.lstm1(input_seq,(h_t,c_t)) #going through the first lstm layer
            h_t2,c_t2=self.lstm2(h_t,(h_t2,c_t2)) #going through the second lstm layer
            otpt=self.linear(h_t2) #fully connected layer
            output+=[otpt]
            
        for i in range(pred_len): #beginning wiht test input and predicting future sequences
            h_t,c_t=self.lstm1(otpt,(h_t,c_t)) #predicting the hidden state through first layer
            h_t2,c_t2=self.lstm2(h_t,(h_t2,c_t2)) #predicting the output prediction
            otpt=self.linear(h_t2) #fully connected layer
            output+=[otpt]
        
        output=torch.stack(output,1).squeeze(2)
        return output

def getData():
    """ Function to load the train and test data
        Returns:
            train_input (torch.Tensor): 95 x 99
            train_target (torch.Tensor): 95 x 99
            test_input (torch.Tensor): 5 x 99
            test_target (torch.Tensor): 5 x 99
    """
    data = torch.load('q4data.pt')
    train_input = torch.from_numpy(data[5:, :-1])
    train_target = torch.from_numpy(data[5:, 1:])
    test_input = torch.from_numpy(data[:5, :-1])
    test_target = torch.from_numpy(data[:5, 1:])
    return train_input, train_target, test_input, test_target

def plotTestResults(pred, epoch, obs_len=999, pred_len=999):
    """ Function to plot the observed and predicted sequence
        Arguments:
            pred (torch.Tensor): 5 x 198
            epoch (int): Epoch number
            obs_len (int): Length of the observed sequence
            pred_len (int): Length of the predicted sequence

    """
    plt.figure(figsize=(32,20))
    colors=['r','b','g','y','cyan','majenta','black','purple','grey','orange'] #list for storing different plot colors
    plt.title("Observed and predicted values for time sequences || epoch: %d"%epoch)
    prediction=pred.data.numpy() #converting tensor to numpy array
    for i in range(pred.size(0)):
        plt.plot(np.arange(obs_len),prediction[i][range(obs_len)],color=colors[i])
        plt.plot(np.arange(obs_len,obs_len+pred_len),prediction[i,range(obs_len,pred_len+obs_len)],color=colors[i],linestyle='--')
    plt.show()
    pass


def main():

    num_epoch = 10
    pred_freq = 1
    pred_len = 999
    
    # Get data
    train_input, train_target, test_input, test_target = getData()

    # Define the network and criterion
    net = LSTMNet()
    net.double()
    criterion = nn.MSELoss()

    # Define the optimizer
    optimizer = optim.LBFGS(net.parameters(), lr=0.8)

    train_loss = []
    test_loss = []

    # Train for 'num_epoch' epochs and test every 'pred_freq' epochs. 
    # When predicting, use pred_len=999
    # Plot test results everytime. At the end, you should have 'num_epoch' number of plots
    
    for i in range(num_epoch):
        print('Epoch:{}'.format(i))
        def closure():
            optimizer.zero_grad()
            out = net(train_input)
            loss = criterion(out, train_target)
            print('loss:{}'.format(loss.item()))
            train_loss.append(loss.item())
            loss.backward()
            return loss
        optimizer.step(closure)
    
        if i%pred_freq == (pred_freq-1) :
            pred = net(test_input, pred_len=pred_len)
            loss = criterion(pred[:, :-pred_len], test_target)
            test_loss.append(loss.item())
            print('Test loss:', loss.item())
            plotTestResults(pred, i)
            

    # TO-DO: Plot training loss v/s epoch
    train_lossavg=[] #list to store mean training loss of each epoch
    train_loss=np.asarray(train_loss)
    for g in range(int(len(train_loss)/20)): #computing mean training loss of each epoch
        train_lossavg.append(np.mean(train_loss[range(g*20,g*20+20)]))
       
    plt.figure()
    print("Training loss v/s epoch")
    plt.plot(np.arange(len(train_lossavg)),train_lossavg)
    plt.title("Traning loss v/s epoch")
    plt.ylabel("Training loss")
    plt.xlabel("epoch")
    plt.show()
    
    # TO-DO: Plot test loss v/s epoch
    plt.figure()
    print("Test loss v/s epoch")
    plt.plot(np.arange(num_epoch),test_loss)
    plt.title("Test loss v/s epoch")
    plt.ylabel("Test loss")
    plt.xlabel("epoch")
    plt.show()


if __name__ == '__main__':

    main()